In [1]:
import time
import numpy as np
import graphviz as gv
from MChain import MChain

In [2]:
%%time
# birth-death process

N = 50
U = np.eye(N) + np.eye(N,k=-1) + np.eye(N,k=1)
U[N-1,N-2] = 0

mc = MChain()
mc.from_matrix(U)

CPU times: user 6.53 ms, sys: 1.12 ms, total: 7.66 ms
Wall time: 5.76 ms


In [3]:
%%time
mc.analyse()

CPU times: user 389 ms, sys: 2.69 ms, total: 392 ms
Wall time: 391 ms


In [4]:
%%time
trace = mc.run(start='s0',steps=10000)

CPU times: user 141 ms, sys: 977 μs, total: 142 ms
Wall time: 140 ms
